In [7]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
# Load the CSV data into a pandas DataFrame
data = pd.read_csv('all_videos_metrics.csv')

X = data.drop(columns=['Deception', 'Video'])
y = data['Deception']

In [4]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Neural Network

In [21]:
# Function to create the model, required for KerasClassifier
def create_model(layers=[100], activation='relu', optimizer='adam', dropout_rate=0.2):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i == 0:
            model.add(Dense(nodes, input_dim=X_train.shape[1], activation=activation))
        else:
            model.add(Dense(nodes, activation=activation))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [14]:
# Wrap Keras model so it can be used by scikit-learn
model = KerasClassifier(model=create_model, epochs=100, batch_size=32)

In [27]:
# Define the parameters range for random search
param_dist = {
    'model__layers': [(10,), (50,), (10, 10), (50, 50)],  # Different layer configurations
    'model__activation': ['relu', 'tanh', 'sigmoid'],
    'model__optimizer': ['adam', 'sgd'],
    'batch_size': [10, 20, 50],
    'epochs': [10, 50, 100]
}
# Create a RandomizedSearchCV object
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=8, cv=2, verbose=2, random_state=42)

In [28]:
# Fit the model
random_search.fit(X_train, y_train)

Fitting 2 folds for each of 8 candidates, totalling 16 fits
Epoch 1/100


c:\Users\cpfit\OneDrive\Documents\GitHub\cv-lie-detection\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


733/733 ━━━━━━━━━━━━━━━━━━━━ 1s 527us/step - accuracy: 0.4933 - loss: 0.7799
Epoch 2/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 536us/step - accuracy: 0.5448 - loss: 0.6944
Epoch 3/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 538us/step - accuracy: 0.5448 - loss: 0.6913
Epoch 4/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 575us/step - accuracy: 0.5481 - loss: 0.6872
Epoch 5/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 1s 882us/step - accuracy: 0.5648 - loss: 0.6839
Epoch 6/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 637us/step - accuracy: 0.5681 - loss: 0.6823
Epoch 7/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 1s 766us/step - accuracy: 0.5635 - loss: 0.6831
Epoch 8/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 619us/step - accuracy: 0.5649 - loss: 0.6835
Epoch 9/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 583us/step - accuracy: 0.5660 - loss: 0.6816
Epoch 10/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 1s 666us/step - accuracy: 0.5666 - loss: 0.6821
Epoch 11/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 580us/step - accuracy: 0.5677 - loss: 0.6828
Epoch 12/100
733/733 ━━━━━━━━━━

c:\Users\cpfit\OneDrive\Documents\GitHub\cv-lie-detection\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


733/733 ━━━━━━━━━━━━━━━━━━━━ 1s 572us/step - accuracy: 0.5203 - loss: 0.7814
Epoch 2/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 561us/step - accuracy: 0.5223 - loss: 0.7020
Epoch 3/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 590us/step - accuracy: 0.5421 - loss: 0.6915
Epoch 4/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 573us/step - accuracy: 0.5600 - loss: 0.6869
Epoch 5/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 587us/step - accuracy: 0.5582 - loss: 0.6870
Epoch 6/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 583us/step - accuracy: 0.5606 - loss: 0.6863
Epoch 7/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 577us/step - accuracy: 0.5610 - loss: 0.6853
Epoch 8/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 576us/step - accuracy: 0.5615 - loss: 0.6852
Epoch 9/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 580us/step - accuracy: 0.5627 - loss: 0.6848
Epoch 10/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 594us/step - accuracy: 0.5592 - loss: 0.6855
Epoch 11/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 580us/step - accuracy: 0.5623 - loss: 0.6848
Epoch 12/100
733/733 ━━━━━━━━━━

...

c:\Users\cpfit\OneDrive\Documents\GitHub\cv-lie-detection\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1831/1831 ━━━━━━━━━━━━━━━━━━━━ 2s 578us/step - accuracy: 0.5376 - loss: 0.7462
Epoch 2/10
1831/1831 ━━━━━━━━━━━━━━━━━━━━ 1s 564us/step - accuracy: 0.5630 - loss: 0.6794
Epoch 3/10
1831/1831 ━━━━━━━━━━━━━━━━━━━━ 1s 577us/step - accuracy: 0.5651 - loss: 0.6767
Epoch 4/10
1831/1831 ━━━━━━━━━━━━━━━━━━━━ 1s 557us/step - accuracy: 0.5701 - loss: 0.6740
Epoch 5/10
1831/1831 ━━━━━━━━━━━━━━━━━━━━ 1s 572us/step - accuracy: 0.5699 - loss: 0.6748
Epoch 6/10
1831/1831 ━━━━━━━━━━━━━━━━━━━━ 1s 559us/step - accuracy: 0.5721 - loss: 0.6726
Epoch 7/10
1831/1831 ━━━━━━━━━━━━━━━━━━━━ 1s 562us/step - accuracy: 0.5714 - loss: 0.6729
Epoch 8/10
1831/1831 ━━━━━━━━━━━━━━━━━━━━ 1s 564us/step - accuracy: 0.5763 - loss: 0.6710
Epoch 9/10
1831/1831 ━━━━━━━━━━━━━━━━━━━━ 1s 556us/step - accuracy: 0.5721 - loss: 0.6717
Epoch 10/10
1831/1831 ━━━━━━━━━━━━━━━━━━━━ 1s 579us/step - accuracy: 0.5708 - loss: 0.6719
1831/1831 ━━━━━━━━━━━━━━━━━━━━ 1s 471us/step
[CV] END batch_size=20, epochs=10, model__activation=relu, model_

c:\Users\cpfit\OneDrive\Documents\GitHub\cv-lie-detection\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1465/1465 ━━━━━━━━━━━━━━━━━━━━ 2s 771us/step - accuracy: 0.5480 - loss: 0.7028
Epoch 2/50
1465/1465 ━━━━━━━━━━━━━━━━━━━━ 1s 764us/step - accuracy: 0.5839 - loss: 0.6607
Epoch 3/50
1465/1465 ━━━━━━━━━━━━━━━━━━━━ 1s 802us/step - accuracy: 0.5965 - loss: 0.6528
Epoch 4/50
1465/1465 ━━━━━━━━━━━━━━━━━━━━ 1s 771us/step - accuracy: 0.6016 - loss: 0.6489
Epoch 5/50
1465/1465 ━━━━━━━━━━━━━━━━━━━━ 1s 784us/step - accuracy: 0.6005 - loss: 0.6491
Epoch 6/50
1465/1465 ━━━━━━━━━━━━━━━━━━━━ 1s 764us/step - accuracy: 0.6001 - loss: 0.6485
Epoch 7/50
1465/1465 ━━━━━━━━━━━━━━━━━━━━ 1s 771us/step - accuracy: 0.6008 - loss: 0.6480
Epoch 8/50
1465/1465 ━━━━━━━━━━━━━━━━━━━━ 1s 779us/step - accuracy: 0.5987 - loss: 0.6464
Epoch 9/50
1465/1465 ━━━━━━━━━━━━━━━━━━━━ 1s 796us/step - accuracy: 0.6029 - loss: 0.6454
Epoch 10/50
1465/1465 ━━━━━━━━━━━━━━━━━━━━ 1s 785us/step - accuracy: 0.6059 - loss: 0.6435
Epoch 11/50
1465/1465 ━━━━━━━━━━━━━━━━━━━━ 1s 765us/step - accuracy: 0.6081 - loss: 0.6445
Epoch 12/50
1465/14

RandomizedSearchCV(cv=2,
                   estimator=KerasClassifier(batch_size=32, epochs=100, model=<function create_model at 0x0000019988C249A0>),
                   n_iter=8,
                   param_distributions={'batch_size': [10, 20, 50],
                                        'epochs': [10, 50, 100],
                                        'model__activation': ['relu', 'tanh',
                                                              'sigmoid'],
                                        'model__layers': [(10,), (50,),
                                                          (10, 10), (50, 50)],
                                        'model__optimizer': ['adam', 'sgd']},
                   random_state=42, verbose=2)

In [29]:
# Print the best parameters and the best score
print("Best parameters:", random_search.best_params_)
print("Best score:", random_search.best_score_)

# Evaluate the best model on the test set
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Test set accuracy:", accuracy)
print("Classification Report:\n", class_report)

Best parameters: {'model__optimizer': 'adam', 'model__layers': (50, 50), 'model__activation': 'tanh', 'epochs': 50, 'batch_size': 50}
Best score: 0.6253654271741209
367/367 ━━━━━━━━━━━━━━━━━━━━ 0s 591us/step
Test set accuracy: 0.6395825364734168
Classification Report:
               precision    recall  f1-score   support

           0       0.58      0.70      0.63      8100
           1       0.71      0.59      0.65     10201

    accuracy                           0.64     18301
   macro avg       0.65      0.65      0.64     18301
weighted avg       0.65      0.64      0.64     18301

